In [1]:
import sqlite3

In [2]:
def init_db():
    conn = sqlite3.connect(":memory:")  # in-memory DB (use "mydb.db" for file)
    #sqlite3.connect(...) → opens a connection to a SQLite database.

    #:memory: → a special name that tells SQLite to create the database in RAM only, not on disk.
    #This means:
    #The database is temporary (lives only while your Python program runs).
    #Once the connection is closed or the program ends, all data is lost.
    
    cursor = conn.cursor()

    # Create tables
    cursor.execute("""
    CREATE TABLE departments (
        dept_id INTEGER PRIMARY KEY AUTOINCREMENT,
        dept_name TEXT NOT NULL,
        location TEXT
    )
    """)

    cursor.execute("""
    CREATE TABLE employees (
        emp_id INTEGER PRIMARY KEY AUTOINCREMENT,
        name TEXT NOT NULL,
        dept_id INTEGER,
        salary REAL,
        FOREIGN KEY (dept_id) REFERENCES departments(dept_id)
    )
    """)

    # Insert sample data
    departments = [
        ("HR", "New York"),
        ("Engineering", "San Francisco"),
        ("Marketing", "Chicago"),
        ("Sales", "UK")
    ]
    cursor.executemany("INSERT INTO departments (dept_name, location) VALUES (?, ?)", departments)
    employees = [
        ("Alice", 1, 50000),
        ("Bob", 2, 70000),
        ("Charlie", 2, 80000),
        ("Diana", 3, 60000),
        ("Eve", 1, 55000),
        ("Test", 7, 55000),
        
    ]
    cursor.executemany("INSERT INTO employees (name, dept_id, salary) VALUES (?, ?, ?)", employees)

    conn.commit()
    return conn

    #conn is your connection object to the SQLite database.

    #.commit() is used to save (persist) changes you made during the session.

    #Without calling commit(), any INSERT, UPDATE, or DELETE you executed will stay in a temporary transaction and might be lost once the connection closes.


    # tHE RETURN HAS TO BE USED IN OTHER FOR THE FUNCTION TO BE USED LATER


In [3]:
def get_all_employees(conn):
    return conn.execute("SELECT * FROM employees").fetchall()

def get_engineering_employees(conn):
    return conn.execute("SELECT name, salary FROM employees WHERE dept_id = 2").fetchall()

def get_statistics(conn):
    return conn.execute("SELECT COUNT(*), AVG(salary), MAX(salary) FROM employees").fetchone()

def add_employee(conn, name, dept_id, salary):
    conn.execute("INSERT INTO employees (name, dept_id, salary) VALUES (?, ?, ?)", (name, dept_id, salary))
    conn.commit()

def update_salary(conn, name, increment):
    conn.execute("UPDATE employees SET salary = salary + ? WHERE name = ?", (increment, name))
    conn.commit()

def delete_employee(conn, name):
    conn.execute("DELETE FROM employees WHERE name = ?", (name,))
    conn.commit()

In [4]:
conn = init_db()


In [5]:
get_all_employees(conn)

[(1, 'Alice', 1, 50000.0),
 (2, 'Bob', 2, 70000.0),
 (3, 'Charlie', 2, 80000.0),
 (4, 'Diana', 3, 60000.0),
 (5, 'Eve', 1, 55000.0),
 (6, 'Test', 7, 55000.0)]

In [ ]:
JOIN
A= [2, 4, 5, 6]
B = [6, 9, 8, 5]
INNER= [5, 6]
OUTER = [2,4,8,9]
LEFT = [[2, 4, 5, 6]
RIGHT = [[6, 9, 8, 5]]

In [6]:
def get_employees_with_departments(conn):
    return conn.execute("""
        SELECT e.name, d.dept_name, d.location, e.salary
        FROM employees e
        INNER JOIN departments d ON e.dept_id = d.dept_id
    """).fetchall()

def get_all_employees_with_optional_dept(conn):
    return conn.execute("""
        SELECT e.name, d.dept_name, d.location
        FROM employees e
        LEFT JOIN departments d ON e.dept_id = d.dept_id
    """).fetchall()

def get_all_departments_with_employees(conn):
    # Simulated RIGHT JOIN using LEFT JOIN with swapped tables
    return conn.execute("""
        SELECT d.dept_name, d.location, e.name
        FROM departments d
        LEFT JOIN employees e ON e.dept_id = d.dept_id
    """).fetchall()

def get_avg_salary_by_department(conn):
    return conn.execute("""
        SELECT d.dept_name, AVG(e.salary) AS avg_salary
        FROM employees e
        INNER JOIN departments d ON e.dept_id = d.dept_id
        GROUP BY d.dept_name
    """).fetchall()


def get_cross_join(conn):
    return conn.execute("""
        SELECT e.name, d.dept_name
        FROM employees e
        CROSS JOIN departments d
        LIMIT 10
    """).fetchall()

    


In [7]:
get_employees_with_departments(conn)

[('Alice', 'HR', 'New York', 50000.0),
 ('Bob', 'Engineering', 'San Francisco', 70000.0),
 ('Charlie', 'Engineering', 'San Francisco', 80000.0),
 ('Diana', 'Marketing', 'Chicago', 60000.0),
 ('Eve', 'HR', 'New York', 55000.0)]

In [8]:
get_all_employees_with_optional_dept(conn)

[('Alice', 'HR', 'New York'),
 ('Bob', 'Engineering', 'San Francisco'),
 ('Charlie', 'Engineering', 'San Francisco'),
 ('Diana', 'Marketing', 'Chicago'),
 ('Eve', 'HR', 'New York'),
 ('Test', None, None)]

In [9]:
get_all_departments_with_employees(conn)

[('HR', 'New York', 'Alice'),
 ('HR', 'New York', 'Eve'),
 ('Engineering', 'San Francisco', 'Bob'),
 ('Engineering', 'San Francisco', 'Charlie'),
 ('Marketing', 'Chicago', 'Diana'),
 ('Sales', 'UK', None)]

In [10]:
get_avg_salary_by_department(conn)

[('Engineering', 75000.0), ('HR', 52500.0), ('Marketing', 60000.0)]

In [11]:
get_cross_join(conn)

[('Alice', 'HR'),
 ('Alice', 'Engineering'),
 ('Alice', 'Marketing'),
 ('Alice', 'Sales'),
 ('Bob', 'HR'),
 ('Bob', 'Engineering'),
 ('Bob', 'Marketing'),
 ('Bob', 'Sales'),
 ('Charlie', 'HR'),
 ('Charlie', 'Engineering')]

In [12]:
conn = init_db()

print("\n--- All Employees ---")
print(get_all_employees(conn))

print("\n--- Engineering Employees ---")
print(get_engineering_employees(conn))

print("\n--- Employee Statistics (count, avg, max) ---")
print(get_statistics(conn))

print("\n--- Add new employee (Frank, Marketing, 65000) ---")
add_employee(conn, "Frank", 3, 65000)
print(get_all_employees(conn))

print("\n--- Update Alice's salary (+5000) ---")
update_salary(conn, "Alice", 5000)
print(get_all_employees(conn))

print("\n--- Delete Eve ---")
delete_employee(conn, "Eve")
print(get_all_employees(conn))

print("\n--- INNER JOIN Employees with Departments ---")
print(get_employees_with_departments(conn))

print("\n--- LEFT JOIN Employees (all employees, dept optional) ---")
print(get_all_employees_with_optional_dept(conn))

print("\n--- RIGHT JOIN Simulation (all departments, employees if any) ---")
print(get_all_departments_with_employees(conn))

print("\n--- Average Salary by Department ---")
print(get_avg_salary_by_department(conn))

print("\n--- CROSS JOIN (first 10 combos) ---")
print(get_cross_join(conn))

conn.close()



--- All Employees ---
[(1, 'Alice', 1, 50000.0), (2, 'Bob', 2, 70000.0), (3, 'Charlie', 2, 80000.0), (4, 'Diana', 3, 60000.0), (5, 'Eve', 1, 55000.0), (6, 'Test', 7, 55000.0)]

--- Engineering Employees ---
[('Bob', 70000.0), ('Charlie', 80000.0)]

--- Employee Statistics (count, avg, max) ---
(6, 61666.666666666664, 80000.0)

--- Add new employee (Frank, Marketing, 65000) ---
[(1, 'Alice', 1, 50000.0), (2, 'Bob', 2, 70000.0), (3, 'Charlie', 2, 80000.0), (4, 'Diana', 3, 60000.0), (5, 'Eve', 1, 55000.0), (6, 'Test', 7, 55000.0), (7, 'Frank', 3, 65000.0)]

--- Update Alice's salary (+5000) ---
[(1, 'Alice', 1, 55000.0), (2, 'Bob', 2, 70000.0), (3, 'Charlie', 2, 80000.0), (4, 'Diana', 3, 60000.0), (5, 'Eve', 1, 55000.0), (6, 'Test', 7, 55000.0), (7, 'Frank', 3, 65000.0)]

--- Delete Eve ---
[(1, 'Alice', 1, 55000.0), (2, 'Bob', 2, 70000.0), (3, 'Charlie', 2, 80000.0), (4, 'Diana', 3, 60000.0), (6, 'Test', 7, 55000.0), (7, 'Frank', 3, 65000.0)]

--- INNER JOIN Employees with Departments -

In [ ]:

import sqlite3

def get_connection():
    conn = sqlite3.connect(":memory:")   # use "university.db" for file DB
    conn.row_factory = sqlite3.Row
    return conn

conn = get_connection()
cur = conn.cursor()

# Create tables
cur.executescript("""
CREATE TABLE Departments (
    dept_id     INTEGER PRIMARY KEY AUTOINCREMENT,
    dept_name   TEXT NOT NULL,
    hod_name    TEXT NOT NULL
);

CREATE TABLE Students (
    student_id  INTEGER PRIMARY KEY AUTOINCREMENT,
    name        TEXT NOT NULL,
    "class"     TEXT NOT NULL,
    dept_id     INTEGER,
    FOREIGN KEY(dept_id) REFERENCES Departments(dept_id)
);

CREATE TABLE Subjects (
    subject_id  INTEGER PRIMARY KEY AUTOINCREMENT,
    subject_name TEXT NOT NULL,
    dept_id     INTEGER,
    FOREIGN KEY(dept_id) REFERENCES Departments(dept_id)
);

CREATE TABLE Marks (
    mark_id     INTEGER PRIMARY KEY AUTOINCREMENT,
    student_id  INTEGER,
    subject_id  INTEGER,
    marks       INTEGER,
    FOREIGN KEY(student_id) REFERENCES Students(student_id),
    FOREIGN KEY(subject_id) REFERENCES Subjects(subject_id)
);
""")

# Insert Sample Data

cur.execute("INSERT INTO Departments (dept_name, hod_name) VALUES ('Computer Science', 'Dr. Smith')")
cur.execute("INSERT INTO Departments (dept_name, hod_name) VALUES ('Mathematics', 'Dr. Johnson')")

students = [
    ("Alice", "Year 1", 1),
    ("Bob", "Year 1", 1),
    ("Charlie", "Year 2", 1),
    ("David", "Year 2", 2),
    ("Eva", "Year 1", 2)
]
cur.executemany("INSERT INTO Students (name, \"class\", dept_id) VALUES (?, ?, ?)", students)

subjects = [
    ("Programming", 1),
    ("Databases", 1),
    ("Algebra", 2),
    ("Statistics", 2)
]
cur.executemany("INSERT INTO Subjects (subject_name, dept_id) VALUES (?, ?)", subjects)

marks = [
    (1, 1, 85), (1, 2, 78), (1, 3, 90), (1, 4, 88),  #Students supposed marks across all IDs and Subjects
    (2, 1, 92), (2, 2, 81), (2, 3, 76), (2, 4, 89),
    (3, 1, 70), (3, 2, 65), (3, 3, 72), (3, 4, 68),
    (4, 1, 88), (4, 2, 92), (4, 3, 80), (4, 4, 85),
    (5, 1, 60), (5, 2, 55), (5, 3, 65), (5, 4, 70)
]
cur.executemany("INSERT INTO Marks (student_id, subject_id, marks) VALUES (?, ?, ?)", marks)
conn.commit()

# Functions
def add_student(name, class_name, dept_id):
    conn.execute("INSERT INTO Students (name, \"class\", dept_id) VALUES (?, ?, ?)", 
                 (name, class_name, dept_id))
    conn.commit()

def add_subject(subject_name, dept_id):
    conn.execute("INSERT INTO Subjects (subject_name, dept_id) VALUES (?, ?)", 
                 (subject_name, dept_id))
    conn.commit()

def add_marks(student_id, subject_id, marks):
    conn.execute("INSERT INTO Marks (student_id, subject_id, marks) VALUES (?, ?, ?)", 
                 (student_id, subject_id, marks))
    conn.commit()

def show_students():
    rows = conn.execute("""
        SELECT s.student_id, s.name, s."class", d.dept_name
        FROM Students s
        JOIN Departments d ON d.dept_id = s.dept_id
    """).fetchall()
    for r in rows:
        print(dict(r))

def show_subjects():
    rows = conn.execute("""
        SELECT sub.subject_id, sub.subject_name, d.dept_name
        FROM Subjects sub
        JOIN Departments d ON d.dept_id = sub.dept_id
    """).fetchall()
    for r in rows:
        print(dict(r))

def show_marks():
    rows = conn.execute("""
        SELECT st.name, sub.subject_name, m.marks
        FROM Marks m
        JOIN Students st ON st.student_id = m.student_id
        JOIN Subjects sub ON sub.subject_id = m.subject_id
    """).fetchall()
    for r in rows:
        print(dict(r))

def student_report(student_id):
    rows = conn.execute("""
        SELECT sub.subject_name, m.marks
        FROM Marks m
        JOIN Subjects sub ON sub.subject_id = m.subject_id
        WHERE m.student_id = ?
    """, (student_id,)).fetchall()

   
def toppers():
    print("Subject-wise toppers:")
    rows = conn.execute("""
        SELECT sub.subject_name, st.name, MAX(m.marks) AS top_score
        FROM Marks m
        JOIN Students st ON st.student_id = m.student_id
        JOIN Subjects sub ON sub.subject_id = m.subject_id
        GROUP BY sub.subject_name
    """).fetchall()
    for r in rows:
        print(dict(r))

    print("\nOverall Topper:")
    row = conn.execute("""
        SELECT st.name, AVG(m.marks) as avg_score
        FROM Marks m
        JOIN Students st ON st.student_id = m.student_id
        GROUP BY st.student_id
        ORDER BY avg_score DESC LIMIT 1
    """).fetchone()
    print(dict(row))

def department_summary():
    rows = conn.execute("""
        SELECT d.dept_name, AVG(m.marks) as avg_marks
        FROM Marks m
        JOIN Students st ON st.student_id = m.student_id
        JOIN Departments d ON d.dept_id = st.dept_id
        GROUP BY d.dept_name
    """).fetchall()
    for r in rows:
        print(dict(r))

def unique_subjects():
    rows = conn.execute("SELECT DISTINCT subject_name FROM Subjects").fetchall()
    return {r["subject_name"] for r in rows}

# -----------------------------
# Demo
# -----------------------------
if __name__ == "__main__":
    print("=== Students ===")
    show_students()

    print("\n=== Subjects ===")
    show_subjects()

    print("\n=== Marks ===")
    show_marks()

    print("\n=== Student Report (1) ===")
    student_report(1)

    print("\n=== Toppers ===")
    toppers()

    print("\n=== Department Summary ===")
    department_summary()

    print("\n=== Unique Subjects ===")
    print(unique_subjects())
import sqlite3

# -----------------------------
# Database Setup
# -----------------------------
def get_connection():
    conn = sqlite3.connect(":memory:")   # use "university.db" for file DB
    conn.row_factory = sqlite3.Row
    return conn

conn = get_connection()
cur = conn.cursor()

# Create tables
cur.executescript("""
CREATE TABLE Departments (
    dept_id     INTEGER PRIMARY KEY AUTOINCREMENT,
    dept_name   TEXT NOT NULL,
    hod_name    TEXT NOT NULL
);

CREATE TABLE Students (
    student_id  INTEGER PRIMARY KEY AUTOINCREMENT,
    name        TEXT NOT NULL,
    "class"     TEXT NOT NULL,
    dept_id     INTEGER,
    FOREIGN KEY(dept_id) REFERENCES Departments(dept_id)
);

CREATE TABLE Subjects (
    subject_id  INTEGER PRIMARY KEY AUTOINCREMENT,
    subject_name TEXT NOT NULL,
    dept_id     INTEGER,
    FOREIGN KEY(dept_id) REFERENCES Departments(dept_id)
);

CREATE TABLE Marks (
    mark_id     INTEGER PRIMARY KEY AUTOINCREMENT,
    student_id  INTEGER,
    subject_id  INTEGER,
    marks       INTEGER,
    FOREIGN KEY(student_id) REFERENCES Students(student_id),
    FOREIGN KEY(subject_id) REFERENCES Subjects(subject_id)
);
""")

# -----------------------------
# Insert Sample Data
# -----------------------------
cur.execute("INSERT INTO Departments (dept_name, hod_name) VALUES ('Computer Science', 'Dr. Smith')")
cur.execute("INSERT INTO Departments (dept_name, hod_name) VALUES ('Mathematics', 'Dr. Johnson')")

students = [
    ("Alice", "Year 1", 1),
    ("Bob", "Year 1", 1),
    ("Charlie", "Year 2", 1),
    ("David", "Year 2", 2),
    ("Eva", "Year 1", 2)
]
cur.executemany("INSERT INTO Students (name, \"class\", dept_id) VALUES (?, ?, ?)", students)

subjects = [
    ("Programming", 1),
    ("Databases", 1),
    ("Algebra", 2),
    ("Statistics", 2)
]
cur.executemany("INSERT INTO Subjects (subject_name, dept_id) VALUES (?, ?)", subjects)

marks = [
    (1, 1, 85), (1, 2, 78), (1, 3, 90), (1, 4, 88),
    (2, 1, 92), (2, 2, 81), (2, 3, 76), (2, 4, 89),
    (3, 1, 70), (3, 2, 65), (3, 3, 72), (3, 4, 68),
    (4, 1, 88), (4, 2, 92), (4, 3, 80), (4, 4, 85),
    (5, 1, 60), (5, 2, 55), (5, 3, 65), (5, 4, 70)
]
cur.executemany("INSERT INTO Marks (student_id, subject_id, marks) VALUES (?, ?, ?)", marks)
conn.commit()

# -----------------------------
# Functions
# -----------------------------
def add_student(name, class_name, dept_id):
    conn.execute("INSERT INTO Students (name, \"class\", dept_id) VALUES (?, ?, ?)", 
                 (name, class_name, dept_id))
    conn.commit()

def add_subject(subject_name, dept_id):
    conn.execute("INSERT INTO Subjects (subject_name, dept_id) VALUES (?, ?)", 
                 (subject_name, dept_id))
    conn.commit()

def add_marks(student_id, subject_id, marks):
    conn.execute("INSERT INTO Marks (student_id, subject_id, marks) VALUES (?, ?, ?)", 
                 (student_id, subject_id, marks))
    conn.commit()

def show_students():
    rows = conn.execute("""
        SELECT s.student_id, s.name, s."class", d.dept_name
        FROM Students s
        JOIN Departments d ON d.dept_id = s.dept_id
    """).fetchall()
    for r in rows:
        print(dict(r))

def show_subjects():
    rows = conn.execute("""
        SELECT sub.subject_id, sub.subject_name, d.dept_name
        FROM Subjects sub
        JOIN Departments d ON d.dept_id = sub.dept_id
    """).fetchall()
    for r in rows:
        print(dict(r))

def show_marks():
    rows = conn.execute("""
        SELECT st.name, sub.subject_name, m.marks
        FROM Marks m
        JOIN Students st ON st.student_id = m.student_id
        JOIN Subjects sub ON sub.subject_id = m.subject_id
    """).fetchall()
    for r in rows:
        print(dict(r))

def student_report(student_id):
    rows = conn.execute("""
        SELECT sub.subject_name, m.marks
        FROM Marks m
        JOIN Subjects sub ON sub.subject_id = m.subject_id
        WHERE m.student_id = ?
    """, (student_id,)).fetchall()

    total = sum(r["marks"] for r in rows)
    avg = total / len(rows)
    grade = "A" if avg >= 70 else "B" if avg >= 50 else "C"
    status = "Pass" if avg >= 50 else "Fail"

    print(f"Report for Student {student_id}")
    for r in rows:
        print(f" {r['subject_name']}: {r['marks']}")
    print(f" Total: {total}, Average: {avg:.2f}, Grade: {grade}, Status: {status}")

def toppers():
    print("Subject-wise toppers:")
    rows = conn.execute("""
        SELECT sub.subject_name, st.name, MAX(m.marks) AS top_score
        FROM Marks m
        JOIN Students st ON st.student_id = m.student_id
        JOIN Subjects sub ON sub.subject_id = m.subject_id
        GROUP BY sub.subject_name
    """).fetchall()
    for r in rows:
        print(dict(r))

    print("\nOverall Topper:")
    row = conn.execute("""
        SELECT st.name, AVG(m.marks) as avg_score
        FROM Marks m
        JOIN Students st ON st.student_id = m.student_id
        GROUP BY st.student_id
        ORDER BY avg_score DESC LIMIT 1
    """).fetchone()
    print(dict(row))

def department_summary():
    rows = conn.execute("""
        SELECT d.dept_name, AVG(m.marks) as avg_marks
        FROM Marks m
        JOIN Students st ON st.student_id = m.student_id
        JOIN Departments d ON d.dept_id = st.dept_id
        GROUP BY d.dept_name
    """).fetchall()
    for r in rows:
        print(dict(r))

def unique_subjects():
    rows = conn.execute("SELECT DISTINCT subject_name FROM Subjects").fetchall()
    return {r["subject_name"] for r in rows}

# -----------------------------
# Demo
# -----------------------------
if __name__ == "__main__":
    print("=== Students ===")
    show_students()

    print("\n=== Subjects ===")
    show_subjects()

    print("\n=== Marks ===")
    show_marks()

    print("\n=== Student Report (1) ===")
    student_report(1)

    print("\n=== Toppers ===")
    toppers()

    print("\n=== Department Summary ===")
    department_summary()

    print("\n=== Unique Subjects ===")
    print(unique_subjects())

